In [161]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
from sophius.db import *
import sqlite3
import pandas as pd

DEVICE_NAME = 'NVIDIA GeForce RTX 4090'

with database:
    database.create_tables([Experiments, Devices, Runs, Models, ModelEpochs])
    
# dev, _ = Devices.get_or_create(name=torch.cuda.get_device_name())
dev, _ = Devices.get_or_create(name=DEVICE_NAME)  

with sqlite3.connect('../data/models.db') as conn:
    # experiments
    exp_df = pd.read_sql('SELECT * FROM experiments', conn)
    exp_df.id += 1
    exp_df.drop(columns='hash', inplace=True)
    for col in ['opt_params', 'sch_params']:
        exp_df[col] = exp_df[col].apply(eval)
    
    # runs and models
    runs_df = pd.read_sql('SELECT * FROM models', conn)
    runs_df.id += 1
    runs_df.exp_id = runs_df.exp_id.astype(int) + 1    
    
    # convert to numeric and round
    numeric_cols = runs_df.columns.tolist()
    numeric_cols.remove('hash')
    for col in numeric_cols:
        runs_df[col] = pd.to_numeric(runs_df[col], errors='coerce')
    for col in ['val_acc', 'train_acc']:
        runs_df[col] = runs_df[col]
    runs_df.time = runs_df.time
    
    models_df = runs_df[['hash', 'flops', 'macs', 'params']].drop_duplicates(subset='hash')
    models_df['id'] = pd.RangeIndex(1, len(models_df) + 1)
    
    runs_df['model_id'] = pd.merge(runs_df, models_df, on='hash', how='left')['id_y']
    runs_df = runs_df[['exp_id', 'model_id', 'val_acc', 'train_acc', 'time']]
    runs_df['device_id'] = dev.id    
    
    # epochs
    epochs_df = pd.read_sql('SELECT * FROM model_epochs', conn)
    for col in epochs_df.columns:
        epochs_df[col] = pd.to_numeric(epochs_df[col], errors='coerce')
    epochs_df.exp_id += 1
    epochs_df.rename(columns={'model_id': 'run_id'}, inplace=True)
    epochs_df.run_id += 1
    
    # round values
    epochs_df.epoch = epochs_df.epoch.astype(int)
    for col in ['val_acc', 'train_acc']:
        epochs_df[col] = epochs_df[col]
    epochs_df.loss = epochs_df.loss
    epochs_df.time = epochs_df.time

In [163]:
# runs_df['model_id'] = runs_df.hash.apply(lambda x: models_df.loc[x, 'id'])
# runs_df.model_id

In [164]:
# runs_df[runs_df.hash.duplicated(keep=False)].sort_values(by='hash')

In [165]:
# model_id = pd.merge(runs_df['hash'], models_df[['hash', 'id']], on='hash')['id']
# runs_df['model_id'] = model_id

In [166]:
runs_df

,exp_id,model_id,val_acc,train_acc,time,device_id
0,1,1,0.663141,0.865986,14.334426,1
1,1,2,0.399900,0.405278,14.214669,1
2,1,3,0.642238,0.700962,14.049893,1
3,1,4,0.479167,0.488632,16.746642,1
4,1,5,0.436639,0.439653,18.272280,1
...,...,...,...,...,...,...
8526,1,8511,0.751863,0.999599,180.764648,1
8527,1,8512,0.748187,0.858253,78.987744,1
8528,1,8513,0.681320,0.969111,54.347447,1
8529,1,8514,0.736849,0.841316,25.946075,1


In [167]:
exp_params = exp_df.to_dict(orient='records')
exp_params

[{'id': 1,
  'val_size': 10000,
  'batch_size': 256,
  'num_epoch': 50,
  'random_seed': 42,
  'optimizer': 'AdamW',
  'opt_params': {'lr': 0.001},
  'scheduler': 'ExponentialLR',
  'sch_params': {'gamma': 0.95},
  'in_shape': '(3, 32, 32)',
  'out_shape': '10'}]

In [168]:
print('Insert experiments')    
for exp_params in exp_df.to_dict(orient='records'):    
    Experiments.insert(**exp_params).on_conflict_ignore().execute()

Insert experiments


In [169]:
epochs_df

,epoch,loss,train_acc,val_acc,time,run_id,exp_id
0,0,240.266655,0.545072,0.529046,0.406046,1,1
1,1,190.269297,0.608474,0.580028,0.708046,1,1
2,2,171.378546,0.645333,0.602364,1.005815,1,1
3,3,159.321967,0.673077,0.615184,1.303349,1,1
4,4,150.305269,0.693209,0.624199,1.601765,1,1
...,...,...,...,...,...,...,...
426545,45,1.176830,0.997396,0.752804,50.567924,8531,1
426546,46,1.000921,0.998097,0.755108,51.625924,8531,1
426547,47,0.872785,0.998297,0.755609,52.685054,8531,1
426548,48,0.770252,0.998197,0.755809,53.740096,8531,1


In [170]:
for exp_params in exp_df.to_dict(orient='records'):
    print(exp_params)


{'id': 1, 'val_size': 10000, 'batch_size': 256, 'num_epoch': 50, 'random_seed': 42, 'optimizer': 'AdamW', 'opt_params': {'lr': 0.001}, 'scheduler': 'ExponentialLR', 'sch_params': {'gamma': 0.95}, 'in_shape': '(3, 32, 32)', 'out_shape': '10'}


In [171]:
Experiments.insert(**exp_params).on_conflict_ignore().execute()

1

In [172]:
from tqdm import tqdm

print('Insert experiments')    
for exp_params in exp_df.to_dict(orient='records'):    
    Experiments.insert(**exp_params).on_conflict_ignore().execute()
    
print('Insert models')
Models.insert_many(models_df.to_dict(orient='records')).on_conflict_ignore().execute()

print('Insert runs')
Runs.insert_many(runs_df.to_dict(orient='records')).on_conflict_ignore().execute()

print('Insert epochs')
for i in tqdm(epochs_df.run_id.unique()):
    mask = epochs_df.run_id == i
    ModelEpochs.insert_many(epochs_df[mask].to_dict(orient='records')).on_conflict_ignore().execute()

Insert experiments
Insert models
Insert runs
Insert epochs


100%|██████████| 8531/8531 [00:52<00:00, 162.81it/s]
